<a href="https://colab.research.google.com/github/iam-fern/Gen_datanew/blob/main/GQtest_cluster_loop_gendatanew_n30_x4_1000rep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Gen_datanew/raw/main/gendatanew_n30_x4_1000rep.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    42.672994  97.324811  ...   69556.240810  9.264830e+05
      1.0    91.008260  32.559338  ...  292108.403561  9.157984e+07
      2.0    91.399967  28.620920  ...  287595.614974  7.791086e+07
      3.0    25.004277  25.789202  ... -145327.102380  8.529675e+05
      4.0    39.270222  21.330987  ...  -62142.773356  4.041818e+06
...                ...        ...  ...            ...           ...
999.0 25.0   37.455632  40.933401  ...  -42370.776485 -1.272674e+06
      26.0   41.089837  87.085643  ...   46054.440667  1.880321e+05
      27.0   26.760841  69.434853  ...  -72236.355966 -2.692535e+05
      28.0   24.340049  57.372145  ... -105735.181502  8.361347e+05
      29.0   99.618164  88.615562  ...  429046.960042 -3.039100e+08

[30000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    42.672994  97.324811  ...   69556.240810  9.264830e+05
      1.0    91.008260  32.559338  ...  292108.403561  9.157984e+07
      2.0    91.399967  28.620920  ...  287595.614974  7.791086e+07
      3.0    25.004277  25.789202  ... -145327.102380  8.529675e+05
      4.0    39.270222  21.330987  ...  -62142.773356  4.041818e+06
...                ...        ...  ...            ...           ...
999.0 25.0   37.455632  40.933401  ...  -42370.776485 -1.272674e+06
      26.0   41.089837  87.085643  ...   46054.440667  1.880321e+05
      27.0   26.760841  69.434853  ...  -72236.355966 -2.692535e+05
      28.0   24.340049  57.372145  ... -105735.181502  8.361347e+05
      29.0   99.618164  88.615562  ...  429046.960042 -3.039100e+08

[30000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,42.672994,97.324811,26.858321,9.958624e+05,9.960392e+05,69556.240810,9.264830e+05
1,0.0,1.0,91.008260,32.559338,71.649414,9.187174e+07,9.187195e+07,292108.403561,9.157984e+07
2,0.0,2.0,91.399967,28.620920,60.283052,7.819826e+07,7.819845e+07,287595.614974,7.791086e+07
3,0.0,3.0,25.004277,25.789202,94.268418,7.074854e+05,7.076404e+05,-145327.102380,8.529675e+05
4,0.0,4.0,39.270222,21.330987,65.129895,3.979540e+06,3.979675e+06,-62142.773356,4.041818e+06
...,...,...,...,...,...,...,...,...,...
29995,999.0,25.0,37.455632,40.933401,87.318043,-1.315221e+06,-1.315045e+06,-42370.776485,-1.272674e+06
29996,999.0,26.0,41.089837,87.085643,47.089224,2.339012e+05,2.340865e+05,46054.440667,1.880321e+05
29997,999.0,27.0,26.760841,69.434853,60.138012,-3.416562e+05,-3.414898e+05,-72236.355966,-2.692535e+05
29998,999.0,28.0,24.340049,57.372145,62.828144,7.302450e+05,7.303995e+05,-105735.181502,8.361347e+05


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    42.672994  97.324811  ...   69556.240810  9.264830e+05
      1.0    91.008260  32.559338  ...  292108.403561  9.157984e+07
      2.0    91.399967  28.620920  ...  287595.614974  7.791086e+07
      3.0    25.004277  25.789202  ... -145327.102380  8.529675e+05
      4.0    39.270222  21.330987  ...  -62142.773356  4.041818e+06
...                ...        ...  ...            ...           ...
999.0 25.0   37.455632  40.933401  ...  -42370.776485 -1.272674e+06
      26.0   41.089837  87.085643  ...   46054.440667  1.880321e+05
      27.0   26.760841  69.434853  ...  -72236.355966 -2.692535e+05
      28.0   24.340049  57.372145  ... -105735.181502  8.361347e+05
      29.0   99.618164  88.615562  ...  429046.960042 -3.039100e+08

[30000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)
    data_sort_clusters = data_clusters.sort_values('clusters')
    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid + 1
    SSR1 = model1.ssr
    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid + 1
    SSR0 = model0.ssr
    MSE0 = model0.mse_resid

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/DF1) / (SSR0/DF0)

    df1 = DF1
    df0 = DF0

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,13.0,11.0,1731841704340679.0,3.5327229790125588e+16,144320142028389.9,3532722979012559.0,24.107493412704198,1.5240396453730654e-06,0.9999992379801773,Reject005=0 : Heteroscedasticity
2.0,9.0,15.0,2431531201616719.0,1.4160215698802646e+16,303941400202089.9,1011443978485903.2,3.494147808439609,0.06348985867573398,0.968255070662133,Reject005=1 : Homoscedasticity
3.0,9.0,15.0,8787190380446831.0,1.8767158178411724e+16,1098398797555853.9,1340511298457980.2,1.2814442864585398,0.724653763351635,0.6376731183241825,Reject005=1 : Homoscedasticity
5.0,12.0,12.0,6953605004079073.0,7138638031075769.0,632145909461733.9,648967093734160.9,1.0266096545443912,0.9644665301253399,0.51776673493733,Reject005=1 : Homoscedasticity
7.0,10.0,14.0,1.7721817440772582e+16,4.13452611793766e+16,1969090826752509.0,3180404706105892.5,1.666439094779038,0.4207354974398818,0.7896322512800591,Reject005=1 : Homoscedasticity
10.0,12.0,12.0,1521216425540211.8,2.9643085470856652e+16,138292402321837.44,2694825951896059.5,19.486435311352786,1.0098078251585818e-05,0.9999949509608742,Reject005=0 : Heteroscedasticity
11.0,12.0,12.0,11238930247647.086,1.108934176714508e+16,1021720931604.2805,1008121978831370.9,986.6901495777746,8.881784197001252e-16,0.9999999999999996,Reject005=0 : Heteroscedasticity
12.0,12.0,12.0,45193215122632.53,4.4494295370136936e+16,4108474102057.503,4044935942739721.5,984.5348521763929,1.1102230246251565e-15,0.9999999999999994,Reject005=0 : Heteroscedasticity
14.0,10.0,14.0,5001401041640908.0,7947679828291878.0,555711226848989.75,611359986791682.9,1.1350647780093017,0.8586008392756992,0.5706995803621504,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,13.0,11.0,1731841704340679.0,3.5327229790125588e+16,144320142028389.9,3532722979012559.0,24.107493412704198,1.5240396453730654e-06,Reject005=0 : Heteroscedasticity
1.0,15.0,9.0,4.182999048025879e+16,1.358526833986735e+16,2987856462875628.0,1698158542483418.8,0.5412890043679215,0.35444288301472154,Reject005=1 : Homoscedasticity
2.0,9.0,15.0,2431531201616719.0,1.4160215698802646e+16,303941400202089.9,1011443978485903.2,3.494147808439609,0.06348985867573398,Reject005=1 : Homoscedasticity
3.0,9.0,15.0,8787190380446831.0,1.8767158178411724e+16,1098398797555853.9,1340511298457980.2,1.2814442864585398,1.275346236648365,Reject005=1 : Homoscedasticity
4.0,12.0,12.0,2.162259730910325e+16,746651965309627.9,1965690664463931.8,67877451391784.35,0.0345310951610463,1.105400168910758e-06,Reject005=0 : Heteroscedasticity
5.0,12.0,12.0,6953605004079073.0,7138638031075769.0,632145909461733.9,648967093734160.9,1.0266096545443912,1.03553346987466,Reject005=1 : Homoscedasticity
6.0,15.0,9.0,6.226854737513042e+16,41093268453827.984,4447753383937887.0,5136658556728.498,0.0010998936645137457,3.945863468254247e-12,Reject005=0 : Heteroscedasticity
7.0,10.0,14.0,1.7721817440772582e+16,4.13452611793766e+16,1969090826752509.0,3180404706105892.5,1.666439094779038,0.4207354974398818,Reject005=1 : Homoscedasticity
8.0,12.0,12.0,1.854849363790246e+16,1.2726004494367848e+16,1686226694354769.0,1156909499487986.2,0.6860936927170845,0.5239543008873699,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    675
Reject005=1 : Homoscedasticity      325
Name: Result_test, dtype: int64